<a href="https://colab.research.google.com/github/ElizabethTeena/Class_PerScholas/blob/main/Final_DB_Project_Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
from prettytable import PrettyTable
import pandas as pd
import re


In [ ]:
def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        print("✅ Connected to database.")
        return conn
    except sqlite3.Error as e:
        print(e)
        return None

In [ ]:
# ✅ Step 2: Create Tables (Schema Design)
def create_tables(conn):
    c = conn.cursor()

    c.execute("""
        CREATE TABLE IF NOT EXISTS Students (
            sid INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            email TEXT UNIQUE NOT NULL,
            password TEXT NOT NULL,
            gpa REAL CHECK(gpa >= 0 AND gpa <= 4)
        );""")

    c.execute("""
        CREATE TABLE IF NOT EXISTS Professors (
            pid INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            department TEXT,
            email TEXT UNIQUE NOT NULL,
            password TEXT NOT NULL
        );""")

    c.execute("""
        CREATE TABLE IF NOT EXISTS Courses (
            cid INTEGER PRIMARY KEY,
            cname TEXT NOT NULL,
            credits INTEGER NOT NULL,
            capacity INTEGER CHECK(capacity <= 25)
        );""")

    c.execute("""
        CREATE TABLE IF NOT EXISTS Prerequisites (
            course_id INTEGER,
            prereq_id INTEGER,
            PRIMARY KEY (course_id, prereq_id),
            FOREIGN KEY (course_id) REFERENCES Courses(cid),
            FOREIGN KEY (prereq_id) REFERENCES Courses(cid)
        );""")

    c.execute("""
        CREATE TABLE IF NOT EXISTS Schedule (
            course_id INTEGER,
            day TEXT,
            start_time TEXT,
            end_time TEXT,
            location TEXT,
            FOREIGN KEY (course_id) REFERENCES Courses(cid)
        );""")

    c.execute("""
        CREATE TABLE IF NOT EXISTS Teaching (
            pid INTEGER,
            cid INTEGER,
            PRIMARY KEY (pid, cid),
            FOREIGN KEY (pid) REFERENCES Professors(pid),
            FOREIGN KEY (cid) REFERENCES Courses(cid)
        );""")

    c.execute("""
        CREATE TABLE IF NOT EXISTS Enrollment (
            sid INTEGER,
            cid INTEGER,
            PRIMARY KEY (sid, cid),
            FOREIGN KEY (sid) REFERENCES Students(sid),
            FOREIGN KEY (cid) REFERENCES Courses(cid)
        );""")
    conn.commit()
    print("✅ Tables created.")

In [ ]:
# ✅ Step 3: Insert Sample Data (for rubric compliance)
def insert_sample_data(conn):
    cur = conn.cursor()

    # Students
    students = [
        (1, "Alice Johnson", "alice@msu.edu", "pass1", 3.5),
        (2, "Bob Smith", "bob@msu.edu", "pass2", 3.0),
        (3, "Charlie Brown", "charlie@msu.edu", "pass3", 2.8),
        (4, "Diana Prince", "diana@msu.edu", "pass4", 3.7),
        (5, "Ethan Hunt", "ethan@msu.edu", "pass5", 3.9)
    ]
    cur.executemany("INSERT OR IGNORE INTO Students VALUES (?, ?, ?, ?, ?)", students)

    # Professors
    professors = [
        (1, "Dr. Xavier", "CS", "xavier@msu.edu", "admin1"),
        (2, "Dr. Banner", "CS", "banner@msu.edu", "admin2"),
        (3, "Dr. Carter", "CS", "carter@msu.edu", "admin3")
    ]
    cur.executemany("INSERT OR IGNORE INTO Professors VALUES (?, ?, ?, ?, ?)", professors)

    # Courses
    courses = [
        (101, "Intro to CS", 3, 25),
        (102, "Data Structures", 4, 25),
        (103, "Algorithms", 4, 25),
        (104, "Computer Networks", 3, 25),
        (105, "Operating Systems", 3, 25)
    ]
    cur.executemany("INSERT OR IGNORE INTO Courses VALUES (?, ?, ?, ?)", courses)

    # Prerequisites
    prerequisites = [
        (102, 101),
        (103, 102)
    ]
    cur.executemany("INSERT OR IGNORE INTO Prerequisites VALUES (?, ?)", prerequisites)

    # Schedule
    schedule = [
        (101, "Mon", "09:00", "10:00", "Room A"),
        (102, "Mon", "09:30", "10:30", "Room B"),  # conflict
        (103, "Tue", "11:00", "12:00", "Room C"),
        (104, "Wed", "13:00", "14:00", "Room D"),
        (105, "Thu", "10:00", "11:00", "Room E")
    ]
    cur.executemany("INSERT OR IGNORE INTO Schedule VALUES (?, ?, ?, ?, ?)", schedule)

    # Teaching assignments
    teaching = [
        (1, 101),
        (1, 102),
        (2, 103),
        (2, 104),
        (3, 105)
    ]
    cur.executemany("INSERT OR IGNORE INTO Teaching VALUES (?, ?)", teaching)
    cur.execute("DELETE FROM Enrollment")

    # Enrollments
    enrollments = [
        (1, 101),
        (2, 101), (2, 103),
        (3, 102), (3, 104),
        (4, 101), (4, 105),
        (5,102),(5, 103), (5, 105)
    ]
    cur.executemany("INSERT OR IGNORE INTO Enrollment VALUES (?, ?)", enrollments)

    conn.commit()
    print("✅ Sample data inserted.")


In [ ]:
# 📧 Step 4: Validation Helpers
def valid_email(email):
    return re.match(r"[^@]+@msu\.edu$", email)

In [ ]:
def time_to_minutes(t):
    h, m = map(int, t.split(":"))
    return h * 60 + m


In [ ]:
# 🔐 Step 5: User Authentication
def authenticate_user(conn, role):
    cur = conn.cursor()
    email = input("Enter email in msu domain (or -1 to register): ")
    if email.strip() == "-1" and role == "student":
        sid = int(input("Enter new Student ID: "))
        name = input("Enter full name: ")
        while True:
          email = input("Enter email (must be in @msu.edu format, or type 'back' to cancel): ").strip()
          if email.lower() == "back":
              return None  # cancels registration and returns to menu
          if valid_email(email):
              break
          print("❌ Invalid email. Please use an @msu.edu email or type 'back'.")

        password = input("Enter password: ")
        while True:
            try:
                gpa = float(input("Enter GPA (0–4): "))
                if 0 <= gpa <= 4:
                    break
                print("❌ GPA must be 0–4.")
            except ValueError:
                print("❌ Invalid GPA.")
        cur.execute("INSERT INTO Students (sid, name, email, password, gpa) VALUES (?, ?, ?, ?, ?)",
                    (sid, name, email, password, gpa))
        conn.commit()
        print("✅ Student created.")
        return (sid, name)
    while not valid_email(email):
        print("❌ Invalid email format.")
        email = input("Enter a valid @msu.edu email (or type 'back' to cancel): ").strip()
        if email.lower() == "back":
            return None

    password = input("Enter password: ")
    query = "SELECT * FROM Professors WHERE email=? AND password=?" if role == "admin" else "SELECT * FROM Students WHERE email=? AND password=?"
    cur.execute(query, (email, password))
    user = cur.fetchone()
    if user:
        print(f"✅ Welcome, {user[1]}!")
        return user
    else:
        print("❌ Invalid credentials.")
        return None


In [ ]:
# 🛠️ Step 6: Admin Menu
import pandas as pd

def admin_menu(conn):
    print("\nAdmin Menu:\n1. Add Course\n2. Edit Course\n3. Delete Course\n4. View Courses\n5. Exit")
    while True:
      try:
        choice = input("Choose an option: ")
        cur = conn.cursor()
        if choice == "1":
            cid = int(input("Course ID: "))
            cname = input("Course Name: ")
            credits = int(input("Credits: "))
            capacity = int(input("Capacity (≤25): "))
            cur.execute("INSERT INTO Courses (cid, cname, credits, capacity) VALUES (?, ?, ?, ?)", (cid, cname, credits, capacity))
            conn.commit()
            print("✅ Course added.")
        elif choice == "2":
            cid = int(input("Course ID to update: "))
            cname = input("New name: ")
            credits = int(input("New credits: "))
            capacity = int(input("New capacity: "))
            cur.execute("UPDATE Courses SET cname=?, credits=?, capacity=? WHERE cid=?", (cname, credits, capacity, cid))
            conn.commit()
            print("✅ Course updated.")
        elif choice == "3":
            cid = int(input("Course ID to delete: "))
            cur.execute("DELETE FROM Courses WHERE cid=?", (cid,))
            conn.commit()
            print("🗑️ Deleted course.")
        elif choice == "4":
            cur.execute("SELECT * FROM Courses")
            rows = cur.fetchall()
            if rows:
                df = pd.DataFrame(rows, columns=["Course ID", "Course Name", "Credits", "Capacity"])
                display(df)
            else:
                print("⚠️ No courses found.")
        elif choice == "5":
            print("👋 Exiting admin menu.")
            break
        else:
            print("❌ Invalid input.")
      except Exception as e:  # Handling any exception during the menu choices
            print(f"An error occurred: {e}")

In [ ]:
# ✅ Step 7: Student Menu with Table Output
def start_student_menu(conn, sid):
    cur = conn.cursor()
    while True:
        print("\nStudent Menu:\nL – List Courses\nE – Enroll\nW – Withdraw\nS – Search\nM – My Classes\nP – Prerequisites\nT – Teaching Professors\nX – Exit")
        choice = input("Enter option: ").upper()

        if choice == "L":
            cur.execute("""
                SELECT DISTINCT C.cid, C.cname, S.day, S.start_time,S.end_time, S.location, P.name
                FROM Courses C
                LEFT JOIN Schedule S ON C.cid = S.course_id
                LEFT JOIN Teaching T ON C.cid = T.cid
                LEFT JOIN Professors P ON T.pid = P.pid
            """)
            rows = cur.fetchall()
            df = pd.DataFrame(rows, columns=["Course ID", "Course Name", "Day", "Start","End", "Location", "Professor"])
            display(df)

        elif choice == "E":
            cid = int(input("Enter Course ID to enroll: "))
            cur.execute("SELECT * FROM Enrollment WHERE sid=? AND cid=?", (sid, cid))
            if cur.fetchone():
                print("❌ Already enrolled.")
                continue
            cur.execute("""
                SELECT prereq_id FROM Prerequisites WHERE course_id=?
                EXCEPT
                SELECT cid FROM Enrollment WHERE sid=?
            """, (cid, sid))
            if cur.fetchall():
                print("❌ Missing prerequisites.")
                continue
            cur.execute("SELECT day, start_time, end_time FROM Schedule WHERE course_id=?", (cid,))
            new_sched = cur.fetchall()
            cur.execute("""
                SELECT S.day, S.start_time, S.end_time
                FROM Enrollment E JOIN Schedule S ON E.cid = S.course_id
                WHERE E.sid=?
            """, (sid,))
            enrolled_sched = cur.fetchall()
            conflict = any(
                d1 == d2 and time_to_minutes(s1) < time_to_minutes(e2) and time_to_minutes(s2) < time_to_minutes(e1)
                for d1, s1, e1 in new_sched for d2, s2, e2 in enrolled_sched
            )

            if conflict:
                print("❌ Schedule conflicting with Another Class you Registered To.")
                continue
            cur.execute("SELECT capacity FROM Courses WHERE cid=?", (cid,))
            cap = cur.fetchone()[0]
            cur.execute("SELECT COUNT(*) FROM Enrollment WHERE cid=?", (cid,))
            if cur.fetchone()[0] >= cap:
                print("❌ Course full.")
                continue
            cur.execute("INSERT INTO Enrollment (sid, cid) VALUES (?, ?)", (sid, cid))
            conn.commit()
            print("✅ Enrolled.")

        elif choice == "W":
            cid = int(input("Enter Course ID to withdraw: "))
            cur.execute("DELETE FROM Enrollment WHERE sid=? AND cid=?", (sid, cid))
            conn.commit()
            print("✅ Withdrawn.")

        elif choice == "S":
            keyword = input("Search course name: ")
            cur.execute("""
                SELECT DISTINCT C.cid, C.cname, S.day, S.start_time, S.location, P.name
                FROM Courses C
                LEFT JOIN Schedule S ON C.cid = S.course_id
                LEFT JOIN Teaching T ON C.cid = T.cid
                LEFT JOIN Professors P ON T.pid = P.pid
                WHERE C.cname LIKE ?
            """, (f"%{keyword}%",))
            rows = cur.fetchall()
            df = pd.DataFrame(rows, columns=["Course ID", "Course Name", "Day", "Start", "Location", "Professor"])
            display(df)

        elif choice == "M":
            cur.execute("""
                SELECT DISTINCT C.cid, C.cname, S.day, S.start_time,S.end_time, S.location, P.name, C.credits
                FROM Enrollment E
                JOIN Courses C ON E.cid = C.cid
                LEFT JOIN Schedule S ON C.cid = S.course_id
                LEFT JOIN Teaching T ON C.cid = T.cid
                LEFT JOIN Professors P ON T.pid = P.pid
                WHERE E.sid=?
            """, (sid,))
            rows = cur.fetchall()
            df = pd.DataFrame(rows, columns=["Course ID", "Course Name", "Day", "Start", "End", "Location", "Professor", "Credits"])
            display(df)
            print("🎓 Total Credits:", df["Credits"].sum())

        elif choice == "P":
            cid = int(input("Enter Course ID: "))
            cur.execute("""
                    SELECT prereq_id, (SELECT cname FROM Courses WHERE cid = prereq_id)
                    FROM Prerequisites
                    WHERE course_id=?
            """, (cid,))
            result = cur.fetchall()
            if result:
                df = pd.DataFrame(result, columns=["Prerequisite ID", "Course Name"])
                display(df)
            else:
                print("📋 No prerequisites.")
        elif choice == "T":
            cur.execute("""
                SELECT P.pid, P.name, P.department, GROUP_CONCAT(C.cname), SUM(C.credits)
                FROM Professors P
                LEFT JOIN Teaching T ON P.pid = T.pid
                LEFT JOIN Courses C ON T.cid = C.cid
                GROUP BY P.pid
            """)
            rows = cur.fetchall()
            df = pd.DataFrame(rows, columns=["Professor ID", "Name", "Department", "Courses Taught", "Total Credits"])
            display(df)

        elif choice == "X":
            print("👋 Goodbye!")
            break
        else:
            print("❌ Invalid option.")


In [ ]:
# 🚀 Step 8: Main Function
def main():
    conn = create_connection("CourseManagement.db")
    # ✅ Create tables before inserting data
    create_tables(conn)
    insert_sample_data(conn)
    print("Welcome to the Course Management System")
    role = input("Login as student or admin (s/a): ").lower()
    if role == 's':
        student = authenticate_user(conn, "student")
        if student:
            start_student_menu(conn, student[0])
    elif role == 'a':
        admin = authenticate_user(conn, "admin")
        if admin:
            admin_menu(conn)
    else:
        print("❌ Invalid role selected.")

main()

✅ Connected to database.
✅ Tables created.
✅ Sample data inserted.
Welcome to the Course Management System
Login as student or admin (s/a): s
Enter email in msu domain (or -1 to register): ethan@msu.edu
Enter password: pass5
✅ Welcome, Ethan Hunt!

Student Menu:
L – List Courses
E – Enroll
W – Withdraw
S – Search
M – My Classes
P – Prerequisites
T – Teaching Professors
X – Exit
Enter option: E
Enter Course ID to enroll: 106
✅ Enrolled.

Student Menu:
L – List Courses
E – Enroll
W – Withdraw
S – Search
M – My Classes
P – Prerequisites
T – Teaching Professors
X – Exit
Enter option: M


,Course ID,Course Name,Day,Start,End,Location,Professor,Credits
0,102,Data Structures,Mon,09:30,10:30,Room B,Dr. Xavier,4
1,103,Algorithms,Tue,11:00,12:00,Room C,Dr. Banner,4
2,105,Operating Systems,Thu,10:00,11:00,Room E,Dr. Carter,3
3,106,IA,None,None,None,None,None,3


🎓 Total Credits: 14

Student Menu:
L – List Courses
E – Enroll
W – Withdraw
S – Search
M – My Classes
P – Prerequisites
T – Teaching Professors
X – Exit
Enter option: X
👋 Goodbye!
